In [1]:
# Ensure compatible versions of libraries are installed
!pip install --upgrade transformers datasets scikit-learn pandas lightgbm numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: pandas
    Found existing ins

In [2]:
import pandas as pd
from IPython.display import display
import os

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)

if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Wed Jun 11 08:37:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
def create_fndg_dataset():
    return pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Datasets/fake_news_dataset_german.csv', sep=',', encoding='utf-8')

def create_fang_covid_dataset():
    return pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Datasets/fang_covid_dataset.csv', sep=',', encoding='utf-8')

fndg_dataset = create_fndg_dataset()
display(fndg_dataset)

fang_covid_dataset = create_fang_covid_dataset()
display(fang_covid_dataset)

,id,url,Titel,Body,Kategorie,Datum,Quelle,Fake,Art
0,773233,http://www.der-postillon.com/2018/01/grokoleak...,Exklusiv! Das geheime WhatsApp-Chat-Protokoll ...,Die Sondierungsgespräche zwischen Union und SP...,wirtschaft,2018-01-18 00:00:00,Postillion,1,NaN
1,773234,http://www.der-postillon.com/2018/01/trump-san...,"Trump droht, jeden zu verspeisen, der an seine...",Nun ist es auch medizinisch offiziell bestätig...,wirtschaft,2018-01-17 00:00:00,Postillion,1,NaN
2,773235,http://www.der-postillon.com/2018/01/fdp-sondi...,"Soli runter, keine Steuererhöhungen, kein Klim...","Es waren zähe Verhandlungen, doch die Freien D...",wirtschaft,2018-01-12 00:00:00,Postillion,1,NaN
3,773236,http://www.der-postillon.com/2018/01/joachim-s...,Hat sie eine Affäre? Joachim Sauer glaubt Ange...,Wo treibt sie sich immer bis spät in die Nacht...,wirtschaft,2018-01-09 00:00:00,Postillion,1,NaN
4,773237,http://www.der-postillon.com/2018/01/halb-so-s...,"""Er hat ja nur HALBneger gesagt"": So begründet...",Der Parteivorstand drückt nochmal ein Auge zu:...,wirtschaft,2018-01-08 00:00:00,Postillion,1,NaN
...,...,...,...,...,...,...,...,...,...
63863,838144,http://www.kleinezeitung.at//international/537...,Lehrer entging durch Hochzeit mit Schülerin Ve...,55-Jähriger muss nach Sex mit damals 15-Jährig...,International,2018-02-26 00:00:00,Kleine,0,NaN
63864,838145,http://www.kleinezeitung.at//wirtschaft/wirtsc...,Warum die Taiwaner Toilettenpapier bunkern,Aus Angst vor Preiserhöhungen bei Klopapier ka...,Wirtschaft,2018-02-26 00:00:00,Kleine,0,NaN
63865,838146,http://www.kleinezeitung.at//wirtschaft/wirtsc...,Warum die Taiwaner Toilettenpapier bunkern,Aus Angst vor Preiserhöhungen bei Klopapier ka...,Wirtschaft,2018-02-26 00:00:00,Kleine,0,NaN
63866,838147,http://www.kleinezeitung.at//wirtschaft/wirtsc...,\r\nDie neue Premium-Klasse von Samsung\r\n ...,Am Vorabend der Eröffnung des Mobile World Con...,Wirtschaft,2018-02-25 00:00:00,Kleine,0,NaN


,Unnamed: 0.1,Unnamed: 0,article,date,header,label,url,hist,tweeet,repl,retw,like,quote
0,39839,39839,Mit einem Kontaktverbot für die Bevölkerung wi...,2021-03-29,"Corona-Krise soll ausgenutzt werden, um Mensch...",fake,https://www.freiewelt.net/nachricht/weltwirtsc...,"[{ ""id"": 1376486728689664000, ""url...",1,0,1,2,0
1,39840,39840,"Dichter zeichnen sich dadurch aus, dass sie ei...",2021-03-29,Vermerke für »Mutations«-Impfungen bereits vor...,fake,https://www.freiewelt.net/nachricht/joe-bidens...,"[{ ""id"": 1376487075999068162, ""url...",1,0,0,1,0
2,39841,39841,Bei den Herstellern von Masken und Masken-Komp...,2021-03-29,Merkel droht mit totaler Sperrung: Machtergrei...,fake,https://www.freiewelt.net/nachricht/merkel-wil...,"[{ ""id"": 1376995922773733378, ""url...",7,0,0,3,0
3,39842,39842,"Schauen wir auf das Leben von Mahatma Gandhi, ...",2021-03-29,Aber nicht genügend Testmöglichkeiten vorhande...,fake,https://www.freiewelt.net/nachricht/berliner-l...,"[{ ""id"": 1376807588977897478, ""url...",6,4,7,14,0
4,39843,39843,Immer wieder denkt man bei Betrachtung der akt...,2021-03-29,AfD-Mitbegründer siegt vor Bundesverfassungsge...,fake,https://www.freiewelt.net/nachricht/oberste-de...,"[{ ""id"": 1376589195150557186, ""url...",1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41237,35122,35122,"Die Seenotrettung wird gestoppt, auch auf Druc...",2020-02-04T06:22:16+01:00,In die Katastrophe stolpern,fake,https://www.contra-magazin.com/2020/02/in-die-...,"[{ ""id"": 1224563996822425600, ""url...",2,0,1,1,0
41238,35123,35123,In Kürze wird China die Welt wirtschaftlich be...,2020-02-04T04:47:20+01:00,Coronavirus: Es wird nicht besser…,fake,https://www.contra-magazin.com/2020/02/coronav...,"[{ ""id"": 1224541470335459329, ""url...",2,1,1,0,0
41239,35124,35124,"Zu viele Patienten, zu wenige Beatmungsgeräte....",2020-02-03T03:44:36+01:00,Thailand: Ärzte „heilen“ Coronavirus-Patienten...,fake,https://www.contra-magazin.com/2020/02/thailan...,"[{ ""id"": 1241025135588622336, ""url...",2,0,2,4,0
41240,35125,35125,"Sollen wir Geld für Konzerne ausgeben, für Alt...",2020-02-03T03:13:30+01:00,Coronavirus: Nicht eingedämmt,fake,https://www.contra-magazin.com/2020/02/coronav...,"[{ ""id"": 1224741441836064769, ""url...",3,0,0,0,0


from matplotlib import pyplot as plt
fang_covid_dataset['Unnamed: 0.1'].plot(kind='hist', bins=20, title='Unnamed: 0.1')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
fang_covid_dataset['Unnamed: 0'].plot(kind='hist', bins=20, title='Unnamed: 0')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
fang_covid_dataset['tweeet'].plot(kind='hist', bins=20, title='tweeet')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
fang_covid_dataset.plot(kind='scatter', x='Unnamed: 0.1', y='Unnamed: 0', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
fang_covid_dataset.plot(kind='scatter', x='Unnamed: 0', y='tweeet', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
fang_covid_dataset.plot(kind='scatter', x='tweeet', y='repl', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
fang_covid_dataset.plot(kind='scatter', x='repl', y='retw', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['date']
  ys = series['Unnamed: 0.1']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = fang_covid_dataset.sort_values('date', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('label')):
  _plot_series(series, series_name, i)
  fig.legend(title='label', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('date')
_ = plt.ylabel('Unnamed: 0.1')

In [32]:
def combine_datasets(fang_covid_dataset, fndg_dataset):
  # Explicitly create a copy of the slice
  fang_covid_dataset_extract = fang_covid_dataset[['header', 'article', 'label']].copy()
  fang_covid_dataset_extract['label'] = fang_covid_dataset_extract['label'].map({'fake': 1, 'real': 0})

  fndg_dataset_extract = fndg_dataset[['Titel', 'Body', 'Fake']].copy()

  fndg_dataset_extract_reduced_renamed = fndg_dataset_extract.rename(columns={
      'Titel': 'header',
      'Body': 'article',
      'Fake': 'label'
  })

  combined_dataset = pd.concat([fang_covid_dataset_extract, fndg_dataset_extract_reduced_renamed], ignore_index=True)

  return combined_dataset

In [33]:
def create_fake_dataset():
  fang_covid_dataset_fake = fang_covid_dataset[fang_covid_dataset['label'] == 'fake']
  fndg_dataset_fake = fndg_dataset[fndg_dataset['Fake'] == 1]

  return combine_datasets(fang_covid_dataset_fake, fndg_dataset_fake)

fake_dataset = create_fake_dataset()
display(fake_dataset)

,header,article,label
0,"Corona-Krise soll ausgenutzt werden, um Mensch...",Mit einem Kontaktverbot für die Bevölkerung wi...,1
1,Vermerke für »Mutations«-Impfungen bereits vor...,"Dichter zeichnen sich dadurch aus, dass sie ei...",1
2,Merkel droht mit totaler Sperrung: Machtergrei...,Bei den Herstellern von Masken und Masken-Komp...,1
3,Aber nicht genügend Testmöglichkeiten vorhande...,"Schauen wir auf das Leben von Mahatma Gandhi, ...",1
4,AfD-Mitbegründer siegt vor Bundesverfassungsge...,Immer wieder denkt man bei Betrachtung der akt...,1
...,...,...,...
17808,Streit um Kostenbeteiligung eskaliert: Werder ...,teilen teilen twittern teilen teilen e-mail \r...,1
17809,Internet erscheint erstmals als Buch,teilen teilen twittern teilen teilen e-mail \r...,1
17810,Keine GroKo-Abstimmungsunterlagen erhalten: Af...,teilen teilen twittern teilen teilen e-mail \r...,1
17811,Trauer in Deutschland: Karin Engelbart gestorben,teilen teilen twittern teilen teilen e-mail \r...,1


In [34]:
def create_real_dataset():
  fang_covid_dataset_real = fang_covid_dataset[fang_covid_dataset['label'] == 'real'].sample(n=14028, random_state=42)
  fndg_dataset_real = fndg_dataset[fndg_dataset['Fake'] == 0].sample(n=14028, random_state=42)

  return combine_datasets(fang_covid_dataset_real, fndg_dataset_real)

real_dataset = create_real_dataset()
display(real_dataset)

,header,article,label
0,Landkreis leistet sich eigenen Baujuristen,Und es gibt ihn doch: den Fasching - wenn auch...,0
1,In Ischgl verbreitet sich das Virus und Tirol ...,Die EU will gemeinsame Kriterien für Schnellte...,0
2,Zwei Plattformen sind eine zu viel,"Die Züge werden voller, die Bahn fährt ihren B...",0
3,Das Coronavirus könnte als schwere Grippewelle...,"Fluglinien, Airports und Logistik-Konzerne bau...",0
4,In der Altersfalle,Als erstes Bundesland hat Thüringen die Kontak...,0
...,...,...,...
28051,Hollande sichert Muslimen den Respekt Frankrei...,Eine Woche nach den islamistischen Anschlägen ...,0
28052,Türkischer Minister will nach Auftrittsverbot ...,Der türkische Wirtschaftsminister Nihat Zeybek...,0
28053,Syrische Rebellen boykottieren Friedensgespräc...,Die syrischen Rebellen lehnen eine Teilnahme a...,0
28054,Kreise: Abe will Japans Pensionsfonds für Wach...,Die japanische Regierung will Insidern zufolge...,0


In [35]:
final_dataset = pd.concat([fake_dataset, real_dataset], ignore_index=True)
display(final_dataset)

,header,article,label
0,"Corona-Krise soll ausgenutzt werden, um Mensch...",Mit einem Kontaktverbot für die Bevölkerung wi...,1
1,Vermerke für »Mutations«-Impfungen bereits vor...,"Dichter zeichnen sich dadurch aus, dass sie ei...",1
2,Merkel droht mit totaler Sperrung: Machtergrei...,Bei den Herstellern von Masken und Masken-Komp...,1
3,Aber nicht genügend Testmöglichkeiten vorhande...,"Schauen wir auf das Leben von Mahatma Gandhi, ...",1
4,AfD-Mitbegründer siegt vor Bundesverfassungsge...,Immer wieder denkt man bei Betrachtung der akt...,1
...,...,...,...
45864,Hollande sichert Muslimen den Respekt Frankrei...,Eine Woche nach den islamistischen Anschlägen ...,0
45865,Türkischer Minister will nach Auftrittsverbot ...,Der türkische Wirtschaftsminister Nihat Zeybek...,0
45866,Syrische Rebellen boykottieren Friedensgespräc...,Die syrischen Rebellen lehnen eine Teilnahme a...,0
45867,Kreise: Abe will Japans Pensionsfonds für Wach...,Die japanische Regierung will Insidern zufolge...,0


In [36]:
final_dataset.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Datasets/final_dataset.csv', index=False)